## Image Captioning Application 

In [1]:
import os
import io
import IPython.display
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']

### The following codes use the ML model named "blip-image-captioning-base"
### This model is trained on 14M parameters and readily available for flurting on huggingface.co

In [2]:
# Helper Fumction 
import requests , json

def get_completion(inputs, parameters = None , ENDPOINT_URL=os.environ['HF_API_ITT_BASE'] ):
    headers = {
        "Authorization": f"Bearer {hf_api_key}",
        "Content-Type" : "application/json"
    }
    data = {"inputs" : inputs}
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",ENDPOINT_URL,headers=headers,data=json.dumps(data))
    return json.loads(response.content.decode("utf-8"))

### Building the  UI using Gradio

In [3]:
import gradio as gr 

def image_to_base64_str(pil_image):
    byte_arr = io.BytesIO()
    pil_image.save(byte_arr, format='PNG')
    byte_arr = byte_arr.getvalue()
    return str(base64.b64encode(byte_arr).decode('utf-8'))

def captioner(image):
    base64_image = image_to_base64_str(image)
    result = get_completion(base64_image)
    return result[0]['generated_text']

gr.close_all()
demo = gr.Interface(fn=captioner,
                    inputs=[gr.Image(label="Upload image", type="pil")],
                    outputs=[gr.Textbox(label="Caption")],
                    title="Image Captioning with BLIP",
                    allow_flagging="never",
                    examples=["cat.jpg", "threecats.jpg", "mountain.jpg"])

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://63510fe300f4a802fd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
